In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
from pynq.lib.video import *
import cv2
import numpy as np

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)
hdmi_in.start()
hdmi_out.start()


height = hdmi_in.mode.height
width = hdmi_in.mode.width

In [2]:
A = np.array([[-1,-1,-1], [-1,8,-1], [-1,-1,-1]], np.uint8) # Filter coefficients - Laplace filter [0,-1,0], [-1,4,-1], [0,-1,0]
gray_frame = np.ndarray(shape=(height,width), dtype=np.uint8)

frame_tmp = np.zeros(shape=(height+2,width+2), dtype=np.uint8)
frame_convoluted = np.zeros(shape=(height+2,width+2), dtype=np.uint8)

out_frame = hdmi_out.newframe()
#out_frame = newframe();
#output_frame = np.ndarray(shape=(height,width), dtype=np.uint8)


In [3]:
numframes = 1

start = time.time()

for _ in range(numframes):
    capture = hdmi_in.readframe()
    for i in range(0,height):   #Color to Gray conversion //height
        for j in range(0,width):  #width
            gray_frame[i,j] = np.uint8(0.07*capture[i,j,2] + 0.72*capture[i,j,1] + 0.21*capture[i,j,0])

    for i in range(1,height+1):   #Color to Gray conversion
        for j in range(1,width+1):
            frame_tmp[i,j] = gray_frame[i-1,j-1]
            
    #for i in range(0,height+2):   
    #    for j in range(0,width+2):
    #        frame_convoluted[i,j] = frame_tmp[i,j]
    #frame_convoluted = map(list, frame_tmp)

    for i in range(1,height+1):   #perform Convolution
        for j in range(1,width+1):
            frame_convoluted[i,j] = np.uint8(A[0,0]*frame_tmp[i-1,j+1] + A[0,1]*frame_tmp[i-1,j] + A[0,2]*frame_tmp[i-1,j-1] +A[1,0]*frame_tmp[i,j+1] + A[1,1]*frame_tmp[i,j]+ A[1,2]*frame_tmp[i,j-1] +A[2,0]*frame_tmp[i+1,j+1] + A[2,1]*frame_tmp[i+1,j] + A[2,2]*frame_tmp[i+1,j-1])

    for i in range(1,height+1):   #function to perform
        for j in range(1,width+1):
            if frame_convoluted[i,j]> 127:
                out_frame[i-1,j-1,0]=255  
                out_frame[i-1,j-1,1]=0
                out_frame[i-1,j-1,2]=0
            else:
                out_frame[i-1,j-1,0]=capture[i-1,j-1,0]  
                out_frame[i-1,j-1,1]=capture[i-1,j-1,1]
                out_frame[i-1,j-1,2]=capture[i-1,j-1,2]

    capture.freebuffer()

    hdmi_out.writeframe(out_frame) #outframe
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

/opt/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in ubyte_scalars


In [4]:
import PIL.Image
img = PIL.Image.fromarray(out_frame)
img.save("/home/xilinx/jupyter_notebooks/Final/opencv_filters.jpg")

In [5]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out